## Baseline

#### Import Library

In [ ]:
# !pip install statsmodels

In [ ]:
# !pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import platform
# from statsmodels.tsa.seasonal import seasonal_decompose # ** 왜 모듈을 못 불러올까요...
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import random

from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import LabelEncoder

from sklearn.cluster import KMeans

# import statsmodels.api as sm # ** 모듈 로드가 안되네요...

from datetime import datetime

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 한글 폰트 설정
if platform.system() == 'Darwin':
    plt.rc('font', family = 'AppleGothic')
else :
    plt.rc('font', family = 'Malgun Gothic')



#### Data Load

In [ ]:
customer = pd.read_csv('./data_in/customers.csv')
location = pd.read_csv('./data_in/locations.csv')
order_items = pd.read_csv('./data_in/order_items.csv')
orders = pd.read_csv('./data_in/orders.csv')
payments = pd.read_csv('./data_in/payments.csv')
product = pd.read_csv('./data_in/products.csv')
review = pd.read_csv('./data_in/reviews.csv')
seller = pd.read_csv('./data_in/sellers.csv')
total_df = pd.read_csv('./data_in/total_df.csv')

### EDA (탐색적 데이터 분석)

#### Top10 제품 카테고리 시각화

In [ ]:
# 카테고리별 주문 수 계산
category_cnt = product['Product_category_name'].value_counts().head(10)

plt.figure(figsize=(12, 8))
bars = category_cnt.sort_values(ascending=True).plot(kind='barh', color='skyblue')

plt.title('Top 10 제품 카테고리')
plt.xlabel('주문 수')
plt.ylabel('제품 카테고리')

# 각 바 위에 값 표시
for bar in bars.containers[0]:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
             f'{int(bar.get_width())}', va='center')

plt.show()

#### 고객 Top 10 도시 시각화

In [ ]:
# 상위 10개 도시 선택
top10_cities = customer['Customer_city'].value_counts().head(10).index

# 상위 10개 도시에 대한 데이터만 필터링
data = customer[customer['Customer_city'].isin(top10_cities)]

# 시각화
plt.figure(figsize=(8,6))
ax = sns.countplot(x='Customer_city', data=data,
                   order=data['Customer_city'].value_counts().index, 
                   palette='icefire_r')

plt.title('Top 10 도시명')
plt.xticks(rotation=45) # 도시 이름이 긴 경우를 위해 x축 라벨 회전

# 각 바 위에 값 표시
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'center', 
                xytext = (0, 10), 
                textcoords = 'offset points')

plt.show()


#### 지불형태 시각화

In [ ]:
# 지불 형태별 개수 계산
category_counts = payments['Payment_type'].value_counts()

# 바 그래프 그리기
fig, ax = plt.subplots(figsize=(12, 7))
bars = ax.bar(category_counts.index, category_counts)

# 각 바에 카운트 값 표시
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

ax.set_xlabel('지불형태')
ax.set_ylabel('갯수')
ax.set_title('지불형태 종류')

plt.show()

#### 월별 매출 시각화

In [ ]:
order = orders.copy()

order['Order_purchase_timestamp'] = pd.to_datetime(order['Order_purchase_timestamp'])
order['YearMonth'] = order['Order_purchase_timestamp'].dt.strftime('%Y%m')

merged_orders = pd.merge(order, order_items, on='Order_id', how='left')
merged_orders['Revenue'] = merged_orders['Price'] * merged_orders['Order_item_id']
monthly_revenue = merged_orders.groupby('YearMonth')['Revenue'].sum().reset_index()

plt.figure(figsize=(10, 6))
sns.lineplot(x='YearMonth', y='Revenue', data=monthly_revenue, marker='o')
plt.title('월별 매출 추이')
plt.xlabel('년월')
plt.ylabel('매출')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
orders_customers = pd.merge(order, customer, on='Customer_id', how='inner')
orders_monthly_active = orders_customers.groupby('YearMonth')['Customer_unique_id'].nunique().reset_index()

orders_monthly_active.head()

#### MAU

In [ ]:
# 바 차트 그리기
plt.figure(figsize=(11, 5))
bars = plt.bar(orders_monthly_active['YearMonth'], orders_monthly_active['Customer_unique_id'], color='skyblue')

# 타이틀 및 라벨 설정
plt.title('월별 활동 유저 : MAU')
plt.xlabel('월')
plt.ylabel('유저')
plt.xticks(rotation=45)

# 각 바 위에 값 표시
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.5, yval, ha='center', va='bottom')

plt.show()

In [ ]:
orders_monthly_active['Monthlychange'] = orders_monthly_active['Customer_unique_id'].pct_change()


filtered_data = orders_monthly_active.query("YearMonth > '201801'")


plt.figure(figsize=(14, 7))
plt.plot(filtered_data['YearMonth'], filtered_data['Monthlychange'], marker='o', linestyle='-', color='blue')
plt.title('월별 변화율', fontsize=15)
plt.xlabel('년월', fontsize=12)
plt.ylabel('변화율', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', linewidth=0.5)
plt.show()

#### PCA 주성분 분석

In [ ]:
# 필요한 라이브러리를 불러옵니다.
# import pandas as pd
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler

# 데이터를 로드합니다.

data = pd.read_csv('./total_df.csv')

# Drop rows where review information is missing **

data_cleaned = data.dropna(subset=['Review_score', 'Review_id', 'Review_creation_date', 'Review_answer_timestamp'])

# Convert product dimension and weight columns to numeric **

dimension_cols = ['Product_weight_g', 'Product_length_cm', 'Product_height_cm', 'Product_width_cm']
data_cleaned[dimension_cols] = data_cleaned[dimension_cols].apply(pd.to_numeric, errors='coerce')

# Drop any rows that could not be converted (if any) **

data_cleaned = data_cleaned.dropna(subset=dimension_cols)

# Convert dates from string to datetime **

date_columns = ['Order_purchase_timestamp', 'Order_delivered_customer_date', 'Review_creation_date', 'Review_answer_timestamp']
for col in date_columns:
    data_cleaned[col] = pd.to_datetime(data_cleaned[col], errors='coerce')

# Calculate derived columns **
# Delivery speed in days **

data_cleaned['Delivery_speed_days'] = (data_cleaned['Order_delivered_customer_date'] - data_cleaned['Order_purchase_timestamp']).dt.days

# Review response time in days **

data_cleaned['Review_response_time_days'] = (data_cleaned['Review_answer_timestamp'] - data_cleaned['Review_creation_date']).dt.days

# Display the new columns and any remaining missing values **

derived_columns_info = data_cleaned[['Delivery_speed_days', 'Review_response_time_days']].describe()
missing_values_updated = data_cleaned.isnull().sum()
derived_columns_info, missing_values_updated


In [ ]:
data_cleaned.info()

In [ ]:
# Order_id 이름 바꾸기

data_cleaned['Order_id'] = data_cleaned['Order_id'].map(
    lambda x: x.replace('ORDER_', '')
)

data_cleaned.head()

In [ ]:
data_cleaned['Product_id'] = data_cleaned['Product_id'].map(
    lambda x: x.replace('PRODUCT_', '')
)

data_cleaned['Customer_id'] = data_cleaned['Customer_id'].map(
    lambda x: x.replace('CUSTOMER_', '')
)

data_cleaned['Review_id'] = data_cleaned['Review_id'].map(
    lambda x: x.replace('REVIEW_', '')
)

In [ ]:
data_cleaned['Seller_id'] = data_cleaned['Seller_id'].map(
    lambda x: x.replace('SELLER_', '')
)

In [ ]:
X = data_cleaned

# 데이터 표준화

scaler = StandardScaler()
X_scaled = pd.DataFrame()


for column in X.columns:
    try:
        # 특정 컬럼에 대해서만 fit_transform을 시도합니다.
        scaled_data = scaler.fit_transform(X[[column]])
        X_scaled[column] = scaled_data.flatten()  # 결과를 DataFrame에 추가합니다.
    except Exception as e:
        # 오류 발생시 해당 컬럼은 제외합니다.
        continue

# PCA 적용

pca = PCA()
pca_result = pca.fit_transform(X_scaled)

# 설명된 분산 비율

explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()




for i in range(0,25,1):
    print(f'{X_scaled.columns[i]}, : {explained_variance[i]}, {cumulative_variance[i]}')


##### 결과 요약 : total_df의 컬럼에 대한 PCA 분석
- 이 과정을 통해, KPI 선정 과정에서, 영향력있는 컬럼을 중심으로 데이터를 분석하고자 함.



- Order_id (16.12% 설명력): 주문 식별자는 데이터셋에서 가장 많은 정보를 포함하는 변수입니다. 주문 관련 다양한 요소들이 이 식별자와 연결되어 있을 수 있습니다.
- Order_item_id (15.28% 설명력): 각 주문에 포함된 아이템의 식별자 역시 상당한 양의 정보를 포함하고 있습니다. 주문된 아이템의 특성과 양이 이 변수를 통해 반영될 수 있습니다.
- Product_id (7.32% 설명력): 제품 식별자는 제품에 대한 중요한 정보를 포함합니다. 제품의 종류와 특성이 이 변수를 통해 데이터에 반영됩니다.
- Seller_id (5.82% 설명력): 판매자 식별자는 판매자의 특성과 판매 패턴을 반영할 수 있으며, 판매자의 다양성과 경쟁 상황을 나타냅니다.
- Price (4.81% 설명력): 제품의 가격은 구매 결정에 큰 영향을 미치며, 고객의 구매력과 제품의 가치를 반영합니다.
- Freight_value (4.22% 설명력): 배송료는 제품의 배송 조건과 비용을 나타내며, 지역적, 물류적 요인을 반영할 수 있습니다.
- Customer_id (4.14% 설명력): 고객 식별자는 특정 고객의 구매 행동과 패턴을 나타내며, 재구매, 고객 충성도 등에 영향을 미칠 수 있습니다.
- Order_purchase_timestamp (4.05% 설명력): 주문이 이루어진 시간은 시즌, 특정 이벤트, 경제 상황 등 다양한 시간적 요소를 반영할 수 있습니다.
- Order_delivered_customer_date (4.00% 설명력): 고객에게 배송 완료된 시간은 배송 속도와 효율성을 나타내며, 고객 만족도와 직접적인 관련이 있을 수 있습니다.
- Seller_zipcode_prefix (3.99% 설명력): 판매자의 지역 코드는 지리적 위치를 반영하며, 물류, 배송 네트워크의 효율성과 연결됩니다.


이 상위 10개 변수는 데이터의 69.75%의 분산을 설명하며, 이를 통해 데이터 세트의 대부분의 정보를 포착할 수 있습니다. 이 변수들은 데이터 분석 및 모델링에서 중요한 요소로 활용될 수 있습니다.

#### TLCC

In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib import font_manager, rc
# import pandas as pd
# import numpy as np
# import platform
# from statsmodels.tsa.seasonal import seasonal_decompose

# 데이터 로드 (예제로 적절한 경로 설정 필요) **
total_df = pd.read_csv('/mnt/data/total_df.csv')

# 날짜 형식으로 변환

total_df['Order_purchase_timestamp'] = pd.to_datetime(total_df['Order_purchase_timestamp'])
total_df['Order_delivered_customer_date'] = pd.to_datetime(total_df['Order_delivered_customer_date'])
total_df['Review_creation_date'] = pd.to_datetime(total_df['Review_creation_date'])
total_df['Review_answer_timestamp'] = pd.to_datetime(total_df['Review_answer_timestamp'])

# 필요한 변수들 생성

total_df['Delivery_speed'] = (total_df['Order_delivered_customer_date'] - total_df['Order_purchase_timestamp']).dt.days
total_df['Review_response_time'] = (total_df['Review_answer_timestamp'] - total_df['Review_creation_date']).dt.days
total_df['Number_of_products_per_order'] = total_df.groupby('Order_id')['Order_item_id'].transform('size')

# 평균 리뷰 점수 계산

total_df['Seller_average_review_score'] = total_df.groupby('Seller_id')['Review_score'].transform('mean')
total_df['Product_average_review_score'] = total_df.groupby('Product_id')['Review_score'].transform('mean')

# 임시로 사용할 변수

total_df['Loyal_Customer_Count'] = np.random.randint(1, 100, size=len(total_df))  # 임의의 값으로 채우기
total_df['Price_competitiveness'] = np.random.rand(len(total_df))  # 임의의 값으로 채우기

# MAU 계산을 위한 Year_Month 컬럼 생성

total_df['Year_Month'] = total_df['Order_purchase_timestamp'].dt.to_period('M')

# MAU 계산

mau = total_df.groupby('Year_Month')['Customer_unique_id'].nunique().reset_index()
mau.columns = ['Year_Month', 'MAU']

# 데이터와 MAU를 병합

total_df = total_df.merge(mau, on='Year_Month', how='left')

# 월별 데이터 평균 계산

monthly_data = total_df.groupby('Year_Month')[['Delivery_speed', 'Review_response_time', 'Number_of_products_per_order',
            'Seller_average_review_score', 'Product_average_review_score',
            'Loyal_Customer_Count', 'Price_competitiveness', 'MAU']].mean()

# 데이터 분해, 계절성 주기를 6개월로 설정

result = seasonal_decompose(monthly_data['MAU'], model='additive', period=6)

# 분해 결과 시각화

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
result.observed.plot(ax=ax1, title='관측치')
result.trend.plot(ax=ax2, title='추세')
result.seasonal.plot(ax=ax3, title='계절성')
result.resid.plot(ax=ax4, title='잔차')
ax1.set_ylabel('MAU')
ax2.set_ylabel('추세')
ax3.set_ylabel('계절성')
ax4.set_ylabel('잔차')
plt.tight_layout()
plt.show()


분석 결과에 대한 자세한 설명은 각 그래프 컴포넌트—관측치(Observed), 추세(Trend), 계절성(Seasonality), 그리고 잔차(Residuals)를 기반으로 해석할 수 있습니다. 이들은 시계열 데이터를 이해하는 데 중요한 요소들입니다:

관측치 (Observed)
이 그래프는 실제 데이터 포인트를 시간에 따라 보여줍니다. 월별 활성 사용자 수(MAU)의 원래 관측된 값들을 시각화한 것입니다.
관측치 그래프를 통해 데이터의 전반적인 변동성과 패턴을 직관적으로 파악할 수 있으며, 주기적 또는 비정기적 변동을 확인할 수 있습니다.
추세 (Trend)
추세 그래프는 데이터의 장기적인 이동 방향을 나타냅니다. 이는 시간이 지남에 따라 증가하거나 감소하는 경향을 보여주며, 일시적인 변동이나 계절적 요인은 제외하고 데이터의 평활화(smoothing)된 형태를 제시합니다.
추세를 통해 비즈니스가 성장하고 있는지, 또는 어떤 문제로 인해 사용자 수가 감소하고 있는지 등의 장기적인 동향을 확인할 수 있습니다.
계절성 (Seasonality)
계절성 그래프는 주기적인 패턴을 보여줍니다. 이는 한 해 동안 또는 설정한 주기(본 예에서는 6개월) 동안 반복되는 패턴을 나타냅니다.
특정 시즌이나 몇 개월 간격으로 반복되는 사용자 활동의 증가 또는 감소를 파악할 수 있습니다. 예를 들어, 휴가 시즌이나 특정 이벤트 기간 동안 사용자 활동이 증가하는 경향을 확인할 수 있습니다.
잔차 (Residuals)
잔차 그래프는 관측치에서 추세와 계절성을 제거한 후 남은 요소를 보여줍니다. 이는 데이터에서 추세와 계절성으로 설명되지 않는 불규칙한 변동을 나타냅니다.
높은 잔차 값은 모델이 데이터의 특정 변동을 적절히 설명하지 못했다는 신호일 수 있으며, 추가적인 요인이나 외부 영향을 고려해야 할 필요가 있습니다.
분석의 활용
이러한 분석을 통해 얻은 정보는 다음과 같은 방법으로 활용할 수 있습니다:

마케팅 및 프로모션 계획: 계절성 분석 결과를 바탕으로 특정 시즌에 맞춘 마케팅 전략을 수립하고 실행할 수 있습니다.
재고 관리 및 리소스 배분: 사용자 활동이 증가하는 시기에 맞춰 재고를 조절하고, 필요 리소스를 효율적으로 배분할 수 있습니다.
사업 전략 및 계획: 추세 분석을 통해 장기적인 사업 전략을 수정하고, 성장 기회를 포착하거나 잠재적 위험을 관리할 수 있습니다.

### Identify Problem

MAU와 매출이 급격히 떨어지는 상황을 문제 상황으로 설정하고, 하락에 영향을 주는 원인들을 조사하기 위해 먼저 MAU와 매출이 급격히 상승했던 원인을 조사한 후 이와 관련이 있을 지 확인해 보기로 했다.

## 매출 상승에 대한 원인 분석

### Refind

** 지금 파일 없어서 나중에 추가

분석 결과 11월 24일 하루에 MAU와 매출이 몰리면서 급격한 상승을 이룬 것으로 확인했고, 이를 블랙 프라이데이에 의한 일시적 상승이라고 판단해 사용자 수와 매출의 하락과 관련이 없는 것으로 결정했다. 이에 사용자 수와 매출에 영향을 줄 수 있는 요인들을 탐색하기 위해 16개의 논문을 선정 후 확인했다.

## 매출 하락에 대한 원인 분석

### 데이터 내에서 매출에 영향을 주는 현재 데이터의 컬럼

#### 데이터 전처리

In [ ]:
# 필요한 라이브러리를 불러옵니다.
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, r2_score
# import random

In [ ]:
# 데이터를 로드합니다.

data_path = './data_in/total_df.csv'
data = pd.read_csv(data_path)

# 결측치를 제거합니다.

data = data.dropna()

# 날짜 관련 컬럼을 datetime 형식으로 변환합니다.

date_columns = ['Order_purchase_timestamp', 'Order_delivered_carrier_date', 'Order_delivered_customer_date', 'Order_estimated_delivery_date', 'Review_creation_date', 'Review_answer_timestamp']
for col in date_columns:
    data[col] = pd.to_datetime(data[col])

#### 파생변수를 여기에 넣으면 좋을 것 같다 **

In [ ]:
# 파생 변수를 생성합니다.

data['Delivery_speed'] = (data['Order_delivered_customer_date'] - data['Order_purchase_timestamp']).dt.days
data['Review_response_time'] = (data['Review_answer_timestamp'] - data['Review_creation_date']).dt.days
customer_order_count = data['Customer_unique_id'].value_counts().to_dict()
data['Customer_order_count'] = data['Customer_unique_id'].apply(lambda x: customer_order_count.get(x, 0))

In [ ]:
# # from datetime import datetime

# # 배송 속도와 리뷰 응답 속도 계산
# data_clean['Order_purchase_timestamp'] = pd.to_datetime(data_clean['Order_purchase_timestamp'])
# data_clean['Order_delivered_customer_date'] = pd.to_datetime(data_clean['Order_delivered_customer_date'])
# data_clean['Review_creation_date'] = pd.to_datetime(data_clean['Review_creation_date'])
# data_clean['Review_answer_timestamp'] = pd.to_datetime(data_clean['Review_answer_timestamp'])

# # 배송 속도 (일 단위)
# data_clean['Delivery_speed'] = (data_clean['Order_delivered_customer_date'] - data_clean['Order_purchase_timestamp']).dt.days

# # 리뷰 응답 속도 (일 단위)
# data_clean['Review_response_time'] = (data_clean['Review_answer_timestamp'] - data_clean['Review_creation_date']).dt.days

# # 기존 특성 확인 및 추가
# data_clean['Number_of_products_per_order'] = data_clean.groupby('Order_id')['Order_item_id'].transform('count')

# # 위의 data_clean transform 된거 확인 해 봐야겠다 count로 되어있네 **

# # 각 판매자와 상품의 평균 리뷰 점수
# data_clean['Seller_average_review_score'] = data_clean.groupby('Seller_id')['Review_score'].transform('mean')
# data_clean['Product_average_review_score'] = data_clean.groupby('Product_id')['Review_score'].transform('mean')

# # 셀러에 대한 고객 충성도 (셀러별 재구매 비율)
# customer_seller_repeated_purchases = data_clean.groupby(['Customer_unique_id', 'Seller_id']).size().reset_index(name='Counts')
# loyal_customers = customer_seller_repeated_purchases[customer_seller_repeated_purchases['Counts'] > 1].groupby('Seller_id').size().reset_index(name='Loyal_Customer_Count')
# data_clean = data_clean.merge(loyal_customers, on='Seller_id', how='left')
# data_clean['Loyal_Customer_Count'] = data_clean['Loyal_Customer_Count'].fillna(0)

# # 각 상품의 평균 가격 및 동일 상품의 가격 경쟁력 (상품별 평균 가격)
# product_price_competitiveness = data_clean.groupby('Product_id')['Price'].mean().reset_index(name='Average_Product_Price')
# data_clean = data_clean.merge(product_price_competitiveness, on='Product_id', how='left')
# data_clean['Price_competitiveness'] = data_clean['Price'] - data_clean['Average_Product_Price']

# # 필요한 데이터 열 선택 및 확인
# data_clean[['Delivery_speed', 'Review_response_time', 'Number_of_products_per_order', 'Seller_average_review_score', 'Product_average_review_score', 'Loyal_Customer_Count', 'Price_competitiveness']].head()


In [ ]:
# 특성과 타겟 변수를 선택합니다.

X = data[['Price', 'Freight_value', 'Review_score', 'Delivery_speed', 'Review_response_time', 'Customer_order_count']]
y = data['Payment_value']

# 데이터를 훈련 세트와 테스트 세트로 분할합니다.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random.randint(1, 100))

# 선형 회귀 모델을 생성하고 훈련합니다.

model = LinearRegression()
model.fit(X_train, y_train)

# 예측을 수행하고 모델을 평가합니다.

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 회귀 계수를 출력합니다.

model.coef_

In [ ]:
# 데이터 정규화

# from sklearn.preprocessing import StandardScaler

# 정규화를 위한 스케일러 초기화

scaler = StandardScaler()

# 훈련 데이터를 정규화

X_train_scaled = scaler.fit_transform(X_train)

# 테스트 데이터를 정규화 (훈련 데이터로 학습된 스케일러 사용)

X_test_scaled = scaler.transform(X_test)

# 정규화된 데이터로 모델 재훈련

model_scaled = LinearRegression()
model_scaled.fit(X_train_scaled, y_train)

# 정규화된 데이터로 예측 및 성능 평가

y_pred_scaled = model_scaled.predict(X_test_scaled)
mse_scaled = mean_squared_error(y_test, y_pred_scaled)
r2_scaled = r2_score(y_test, y_pred_scaled)

# 결과 출력

print('Normalized MSE:', mse_scaled)
print('Normalized R^2 Score:', r2_scaled)
print('Normalized Coefficients:', model_scaled.coef_)


In [ ]:
# 모델 성능과 계수를 출력합니다.

print('MSE:', mse)
print('R^2 Score:', r2)
print('Coefficients:', model.coef_)

#### 결과 해석

### DAU ** (변수와 인풋 데이터 바꾼 후 다시 검증 필요) + 주석 한글

In [ ]:
# import pandas as pd

# 데이터 파일 불러오기
data_path = './data_in/total_df.csv'
data = pd.read_csv(data_path)

# 데이터의 첫 몇 행 확인
data.head()

#### 데이터 전처리 **

In [ ]:
# 결측치 및 데이터 타입 확인
data_info = data.info()
missing_values = data.isnull().sum()

data_info, missing_values[missing_values > 0]


In [ ]:
# 리뷰 관련 결측치 제거
data_clean = data.dropna(subset=['Review_id', 'Review_score', 'Review_creation_date', 'Review_answer_timestamp'])

# 데이터 타입 변환
columns_to_convert = ['Product_weight_g', 'Product_length_cm', 'Product_height_cm', 'Product_width_cm']
data_clean[columns_to_convert] = data_clean[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# 변환 후 결측치 다시 확인
data_clean.info()
additional_missing = data_clean[columns_to_convert].isnull().sum()
data_clean.head(), additional_missing


In [ ]:
# 추가 결측치 제거
data_clean = data_clean.dropna()

# 최종 데이터 확인
final_data_info = data_clean.info()
final_sample = data_clean.head()
final_data_info, final_sample


In [ ]:
# from datetime import datetime

# 배송 속도와 리뷰 응답 속도 계산
data_clean['Order_purchase_timestamp'] = pd.to_datetime(data_clean['Order_purchase_timestamp'])
data_clean['Order_delivered_customer_date'] = pd.to_datetime(data_clean['Order_delivered_customer_date'])
data_clean['Review_creation_date'] = pd.to_datetime(data_clean['Review_creation_date'])
data_clean['Review_answer_timestamp'] = pd.to_datetime(data_clean['Review_answer_timestamp'])

# 배송 속도 (일 단위)
data_clean['Delivery_speed'] = (data_clean['Order_delivered_customer_date'] - data_clean['Order_purchase_timestamp']).dt.days

# 리뷰 응답 속도 (일 단위)
data_clean['Review_response_time'] = (data_clean['Review_answer_timestamp'] - data_clean['Review_creation_date']).dt.days

# 기존 특성 확인 및 추가
data_clean['Number_of_products_per_order'] = data_clean.groupby('Order_id')['Order_item_id'].transform('count')

# 위의 data_clean transform 된거 확인 해 봐야겠다 count로 되어있네 **

# 각 판매자와 상품의 평균 리뷰 점수
data_clean['Seller_average_review_score'] = data_clean.groupby('Seller_id')['Review_score'].transform('mean')
data_clean['Product_average_review_score'] = data_clean.groupby('Product_id')['Review_score'].transform('mean')

# 셀러에 대한 고객 충성도 (셀러별 재구매 비율)
customer_seller_repeated_purchases = data_clean.groupby(['Customer_unique_id', 'Seller_id']).size().reset_index(name='Counts')
loyal_customers = customer_seller_repeated_purchases[customer_seller_repeated_purchases['Counts'] > 1].groupby('Seller_id').size().reset_index(name='Loyal_Customer_Count')
data_clean = data_clean.merge(loyal_customers, on='Seller_id', how='left')
data_clean['Loyal_Customer_Count'] = data_clean['Loyal_Customer_Count'].fillna(0)

# 각 상품의 평균 가격 및 동일 상품의 가격 경쟁력 (상품별 평균 가격)
product_price_competitiveness = data_clean.groupby('Product_id')['Price'].mean().reset_index(name='Average_Product_Price')
data_clean = data_clean.merge(product_price_competitiveness, on='Product_id', how='left')
data_clean['Price_competitiveness'] = data_clean['Price'] - data_clean['Average_Product_Price']

# 필요한 데이터 열 선택 및 확인
data_clean[['Delivery_speed', 'Review_response_time', 'Number_of_products_per_order', 'Seller_average_review_score', 'Product_average_review_score', 'Loyal_Customer_Count', 'Price_competitiveness']].head()


In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, r2_score
# import numpy as np
# import random

# 데이터 준비
features = ['Delivery_speed', 'Review_response_time', 'Number_of_products_per_order', 
            'Seller_average_review_score', 'Product_average_review_score', 
            'Loyal_Customer_Count', 'Price_competitiveness'] # - 전체 데이터 컬럼으로 수정 **
X = data_clean[features]
y = data_clean['Payment_value']  # 예를 들어, 주문당 결제한 가격을 결과 변수로 사용 - 이부분 수정 **

# 데이터 분할: 트레인 80%, 테스트 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random.randint(1, 100)

# 선형 회귀 모델 구축
model = LinearRegression()
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 모델 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, r2


선형 회귀분석시 r2값이 낮아 추출한 특성들의 설명력이 부족하다고 생각했고, ** (`Payment_value` 수정시 변화 가능)
DAU라는 복잡한 결과를 설명하기에 추출한 특성이 부족하다고 판단해 특성들을 추가하여 다시 선형 회귀분석 진행 ** (`Payment_value` 수정시 변화 가능)

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# 주문 개수(고객별)
data_clean['Orders_count'] = data_clean.groupby('Customer_unique_id')['Order_id'].transform('nunique')

# 더미 변수 생성
data_clean['Order_status_completed'] = (data_clean['Order_status'] == 'delivered').astype(int)
data_clean['Payment_type_encoded'] = LabelEncoder().fit_transform(data_clean['Payment_type'])
data_clean['Customer_city_encoded'] = LabelEncoder().fit_transform(data_clean['Customer_city'])

# 추가적으로 고려할 특성
additional_features = ['Order_status_completed', 'Payment_type_encoded', 'Customer_city_encoded', 'Orders_count']

# 기존 특성과 합치기
all_features = features + additional_features
X = data_clean[all_features]
y = data_clean['Payment_value']  # 예를 들어, 주문당 결제한 가격을 결과 변수로 사용 **

# 데이터 분할: 트레인 80%, 테스트 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=ramdom.randint(1, 100)

# 선형 회귀 모델 구축
model = LinearRegression()
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 모델 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, r2


#### 다른 선형 회귀 분석 (**비교 필요)

In [ ]:
data = pd.read_csv('./total_df.csv')

In [ ]:
# 일일 구매 고객 수 계산

data['Date'] = data['Order_purchase_timestamp'].dt.date
daily_customers = data.groupby('Date')['Customer_id'].nunique().rename('Daily_customers')
data = data.join(daily_customers, on='Date')

# 이미 만든 컬럼들과 일일 구매 고객 수를 포함하여 선형 회귀 모델 훈련

features = ['Avg_category_price', 'Price_competitiveness', 'Avg_seller_review',
            'Avg_product_review', 'Customer_loyalty', 'Items_per_order', 'Total_payment_per_order',
            'Delivery_speed', 'Review_response_time']
target = 'Daily_customers'

# 데이터셋을 훈련 세트와 테스트 세트로 분리

X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 선형 회귀 모델 초기화 및 훈련

model = LinearRegression()
model.fit(X_train, y_train)

# 모델 평가

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# 계수 출력

coefficients = pd.DataFrame(model.coef_, index=features, columns=['Coefficient'])

coefficients, rmse

특성 추가 후 분석했으나 설명력이 낮은 것을 확인. DAU를 결정하는 것에는 많은 요소가 복합적으로 얽혀있고, 이를 ** (`Payment_value` 수정시 변화 가능)
선형 분석 모델로 판단하기 어렵다고 생각해 무작위성이 포함된 모델인 랜덤 포레스트로 모델 변경 ** (`Payment_value` 수정시 변화 가능)

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# 랜덤 포레스트 회귀 모델 구축
rf_model = RandomForestRegressor(n_estimators=200, random_state=random.randint(1, 100))  # 200개의 트리 사용, 무작위 시드 적용
rf_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred_rf = rf_model.predict(X_test)

# 모델 평가
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

mse_rf, r2_rf


시드를 무작위화 하여 각 결과값이 다를 수 있지만, 평균적으로 약 0.66의 r2값을 확인할 수 있고,
선형 회귀 분석 모델보다 낮은 mse를 확인해 더 적합한 모델인 것을 확인했다.
추출한 특성들이 DAU와 유의미한 연관성을 갖고 있다는 것을 확인했다. ** (`Payment_value` 수정시 변화 가능)

### 고객 세그먼트 분석 ** 주석 한글로 수정 필요

In [ ]:
# import pandas as pd

# Load the data from the uploaded CSV file **

data_path = './total_df.csv'
data = pd.read_csv(data_path)

In [ ]:
# from sklearn.preprocessing import StandardScaler, LabelEncoder

# Filter out customer-related columns

customer_features = data[['Customer_city', 'Customer_state', 'Payment_type', 'Payment_installments', 'Payment_value', 'Review_score']]

# Handling missing values by dropping rows with any NaNs

customer_features_cleaned = customer_features.dropna()

# Encoding categorical data

label_encoders = {}
for column in ['Customer_city', 'Customer_state', 'Payment_type']:
    le = LabelEncoder()
    customer_features_cleaned[column] = le.fit_transform(customer_features_cleaned[column])
    label_encoders[column] = le

# Scale the numerical data

scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features_cleaned[['Payment_installments', 'Payment_value', 'Review_score']])

# Combine encoded and scaled data into one dataframe for clustering

# import numpy as np

encoded_features = customer_features_cleaned[['Customer_city', 'Customer_state', 'Payment_type']].values
prepared_features = np.concatenate([encoded_features, scaled_features], axis=1)

prepared_features.shape


In [ ]:
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt

# 데이터를 준비하는 단계
# prepared_features는 전처리된 특성 데이터를 포함해야 함

# Using the Elbow Method to find the optimal number of clusters

wcss = []  # Within-cluster sum of squares
for i in range(1, 11):  # 예를 들어 클러스터의 개수를 1부터 10까지 테스트
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=random.randint(1, 100))
    kmeans.fit(prepared_features)
    wcss.append(kmeans.inertia_)

# Plotting the results onto a line graph to observe the 'elbow'

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method For Optimal Number of Clusters')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')  # Within-cluster sums of squares
plt.grid(True)
plt.show()


#### 결과 분석 **

### LTV ** 분석할 컬럼 추가 및 회귀 분석 y값 수정 필요

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error

In [ ]:
# ### 1. 데이터 전처리 ↓



# # 데이터 불러오기
# data = pd.read_csv('total_df.csv')

# # 날짜 컬럼을 datetime 형식으로 변환
# date_columns = ['Order_purchase_timestamp', 'Order_delivered_carrier_date', 'Order_delivered_customer_date',
#                 'Order_estimated_delivery_date', 'Review_creation_date', 'Review_answer_timestamp']
# for col in date_columns:
#     data[col] = pd.to_datetime(data[col])

# # 1. 판매하는 품목의 평균 가격대
# category_price_avg = data.groupby('Product_category_name')['Price'].mean().rename('Avg_category_price')
# data = data.join(category_price_avg, on='Product_category_name')

# # 2. 동일한 상품에 대한 가격 경쟁력
# data['Price_competitiveness'] = data['Price'] - data['Avg_category_price']

# # 3. 판매자의 평균 리뷰 점수
# seller_review_avg = data.groupby('Seller_id')['Review_score'].mean().rename('Avg_seller_review')
# data = data.join(seller_review_avg, on='Seller_id')

# # 4. 상품의 평균 리뷰 점수
# product_review_avg = data.groupby('Product_id')['Review_score'].mean().rename('Avg_product_review')
# data = data.join(product_review_avg, on='Product_id')

# # 5. 셀러에 대한 고객 충성도 (고유 고객 수 대비 반복 구매 비율)
# repeat_customers = data.groupby('Seller_id')['Customer_id'].value_counts().gt(1).groupby('Seller_id').mean().rename('Customer_loyalty')
# data = data.join(repeat_customers, on='Seller_id')

# # 6. 한 주문에 포함된 상품의 개수
# items_per_order = data.groupby('Order_id')['Order_item_id'].max().rename('Items_per_order')
# data = data.join(items_per_order, on='Order_id')

# # 7. 한 주문 당 결제한 가격
# total_payment_per_order = data.groupby('Order_id')['Payment_value'].sum().rename('Total_payment_per_order')
# data = data.join(total_payment_per_order, on='Order_id')

# # 8. 배송 속도 (주문 날짜부터 고객이 상품을 받은 날짜까지의 일수)
# data['Delivery_speed'] = (data['Order_delivered_customer_date'] - data['Order_purchase_timestamp']).dt.days

# # 9. 리뷰 응답 속도 (리뷰 작성일부터 판매자 응답일까지의 시간)
# data['Review_response_time'] = (data['Review_answer_timestamp'] - data['Review_creation_date']).dt.days


In [ ]:
### 2. 랜덤포레스트!!


    # 고객 당 총 결제 금액 계산
    #customer_total_spending = data.groupby('Customer_id')['Payment_value'].sum().rename('Total_spending')

    # 데이터 프레임에 결제 금액 추가
    #data = data.join(customer_total_spending, on='Customer_id')

# 필요한 특성 선택

features = [
    'Avg_category_price', 'Price_competitiveness', 'Avg_seller_review', 'Avg_product_review',
    'Customer_loyalty', 'Items_per_order', 'Delivery_speed', 'Review_response_time'
] # ** total_df의 다른 컬럼 추가가 가능 한 지 확인, 안된다면 가능한 컬럼들만 추가하면 될 것 같아요
X = data[features]
y = data['Total_payment_per_order'] # ** 주문당 결제 금액으로 계산하는 것이 아닌 고객당 총 결제 금액으로 계산 필요

# 데이터 분할

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random.randint(1, 100))

# 모델 생성 및 훈련

model = RandomForestRegressor(n_estimators=100, random_state=random.randint(1, 100))
model.fit(X_train, y_train)

# 예측 및 성능 평가

predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print('Mean Squared Error:', mse)

# 변수 중요도

feature_importances = model.feature_importances_
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
importance_df.sort_values(by='Importance', ascending=False, inplace=True)

In [ ]:
importance_df

### ASP ** 다른 모델 선택 필요

In [ ]:
# import pandas as pd

# Load the provided CSV file

data = pd.read_csv('./total_df.csv')

# Calculate the correlation between 'Price' (as ASP) and 'Payment_value' (as an approximation of net profit)

correlation = data[['Price', 'Payment_value']].corr() # ** 상품 가격과 지불 가격의 상관관계가 아닌 아래 feature들과 `Avg_payment_per_order`간의 corr 필요

# Print the correlation

print(correlation)


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, r2_score

# 데이터 로드

data = pd.read_csv('./data_in/total_df.csv')

# 관심 있는 특성 컬럼 선택

features_columns = [
    'Price', 'Freight_value', 'Product_weight_g', 'Product_length_cm', 'Product_height_cm', 'Product_width_cm',
    'Payment_installments', 'Review_score', 'Avg_category_price', 'Price_competitiveness', 'Avg_seller_review',
    'Avg_product_review', 'Customer_loyalty', 'Items_per_order', 'Delivery_speed', 'Review_response_time'
]

# 데이터 클리닝: 'Unknown' 값을 NaN으로 변환 후 제거

data_cleaned = data[features_columns + ['Avg_payment_per_order']].replace('Unknown', np.nan).dropna()

# 특성(X)과 타겟(y) 변수 설정

X = data_cleaned[features_columns]
y = data_cleaned['Avg_payment_per_order']

# 데이터를 훈련 세트와 테스트 세트로 분리

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random.randint(1, 100))

# 선형 회귀 모델 초기화 및 학습

model = LinearRegression()
model.fit(X_train, y_train)

# 테스트 세트에 대한 예측

y_pred = model.predict(X_test)

# 모델 평가: 결정 계수(R²)와 평균 제곱근 오차(RMSE) 계산

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# 회귀 계수 출력

coefficients = pd.DataFrame({
    'Feature': features_columns,
    'Coefficient': model.coef_
})

# 결과 출력

print(coefficients)
print(f'결정 계수(R²): {r2}')
print(f'평균 제곱근 오차(RMSE): {rmse}')

#### 결과 분석 ** (모델의 설명력이 낮기에 다른 모델을 찾아야 할 것 같아요)

모델 성능: 결정 계수(R²) 값이 0.436으로 나타나, 모델이 데이터의 약 43.6%만 설명하고 있음을 알 수 있습니다. 이는 모델이 개선될 여지가 있음을 시사합니다.

중요 변수:
Items_per_order는 매우 높은 양의 계수를 가지고 있어, 주문당 많은 아이템 수가 주문당 평균 지불 금액을 증가시키는 중요한 요인임을 나타냅니다.

Customer_loyalty와 Avg_seller_review는 높은 음의 계수를 가지고 있어, 이들 변수가 높을수록 주문당 평균 지불 금액이 감소함을 나타냅니다. 이는 충성도가 높은 고객이나 높은 평가를 받는 판매자가 더 낮은 가격을 제공할 수 있음을 의미할 수 있습니다.

비즈니스 의사 결정: 이 분석을 통해 얻은 통찰력은 마케팅 전략, 가격 책정 전략, 고객 관리 전략 등을 조정하는 데 사용될 수 있습니다. 또한, 추가적인 변수를 모델에 포함시키거나 다른 유형의 모델을 시도하여 예측 성능을 높일 수 있습니다.

### 상품 다양성 ** corr 외의 다른 모델 (선형 회귀 분석과 같은) 추가하는 것이 필요

In [ ]:
# 라이브러리 로드

# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

In [ ]:
# products.csv 로드

products_path = './data_in/products.csv'
products = pd.read_csv(products_path)

# 주문당 제품 카테고리 확인 위해 order_items.csv와 products.csv 병합

order_products_df = pd.merge(order_items, products, on='Product_id', how='left')

# order_products_df.head()

#### 2. 판매자 별 상품의 종류 수, 취급하는 카테고리 수와 판매자 총 매출의 연관성 확인

In [ ]:
# Seller_id로 그룹화 하여 상품 명과 카테고리 집계 

seller_analysis = order_products_df.groupby('Seller_id').agg(
    Unique_Product_Count=('Product_id', 'nunique'),
    Unique_Category_Count=('Product_category_name', 'nunique'),
    # 총 매출 계산
    Total_Revenue=('Revenue', 'sum')
).reset_index()

# seller_analysis.head()

# 상품명, 카테고리 수와 총 매출간의 연관성 계산

correlation_matrix = seller_analysis[['Unique_Product_Count', 'Unique_Category_Count', 'Total_Revenue']].corr()

# 상관관계 시각화를 위한 히트맵

plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation between Product Diversity, Category Diversity, and Revenue')
plt.show()


#### 3. 결과 분석 (상관관계)
- 상품의 종류와 매출의 상관관계 : 약 0.65
- 카테고리의 종류와 매출의 상관관계 : 약 0.31
- cf. 상품의 종류와 카테고리 종류의 상관관계 : 약 0.48

### 카테고리 집중도 (** LTV 분석, PCA 분석 결과로 나온 내용인데 다른 작업 먼저 하고 나중에 추가하면 될 것 같아요)

#### 판매량이 높거나 매출이 높은 카테고리에(또는, bed table과 같은 홈 가구 품목만을 집중적으로 다루는 것이) 집중하는 것이 다른 카테고리들의 판매량을 끌어 올리는 것에 집중하는 것 보다 단기적, 장기적 매출 상승에 도움이 될 지

### 가격 경쟁력 *** 위 corr과 아래 회귀분석 연결 및 순서 배열 정도만 수정 필요

#### 1. 동일한 상품이 여러 명에게 취급되는 지 여부 확인

In [ ]:
# 동일 상품을 다양한 판매자가 취급하는 지 확인

multi_seller_products = order_items.groupby('Product_id').nunique()

# 한 판매자에게서만 판매되지 않는 상품들을 변수로 저장

products_sold_by_multiple_sellers = multi_seller_products[multi_seller_products['Seller_id'] > 1]

# 다양한 판매자가 취급하는 상품의 수 확인

count_products_multiple_sellers = len(products_sold_by_multiple_sellers)

count_products_multiple_sellers, products_sold_by_multiple_sellers.reset_index().head()


#### 2. 상품을 평균 가격보다 저렴하게 판매할 경우 Revenue에 미치는 영향을 분석

In [ ]:
# 다양한 판매자에게 판매되는 상품으로 `order_items` 데이터를 필터링 후 변수에 저장

multi_seller_order_items = order_items[order_items['Product_id'].isin(products_sold_by_multiple_sellers.index)]

# 다양한 판매자들에게 판매되는 상품의 평균 가격을 계산

average_price_multi_seller_products = multi_seller_order_items.groupby('Product_id').agg(
    Average_Price=('Price', 'mean')
).reset_index()

# 상품의 가격을 평균가와 비교하기 위해 평균 가격을 다시 `multi_seller_order_items`와 병합

multi_seller_price_comparison = pd.merge(multi_seller_order_items, average_price_multi_seller_products, on='Product_id', how='left')

# 평균보다 낮은 가격의 상품들을 변수로 저장

multi_seller_price_comparison['Price_Below_Average'] = multi_seller_price_comparison['Price'] < multi_seller_price_comparison['Average_Price']

# 해당 상품들의 매출을 계산

multi_seller_price_comparison['Revenue'] = multi_seller_price_comparison['Price'] * multi_seller_price_comparison['Order_item_id']

# 매출을 비교하기 위해 `Product_id`와 `Price_Below_Average`로 그룹화

product_revenue_comparison_multi_seller = multi_seller_price_comparison.groupby(['Product_id', 'Price_Below_Average']).agg(
    Average_Revenue=('Revenue', 'mean')
).reset_index()

# 매출과 평균가보다 낮게 판매되는 상품들의 가격의 상관계수 계산

correlation_price_revenue_multi_seller = product_revenue_comparison_multi_seller['Average_Revenue'].corr(
    product_revenue_comparison_multi_seller['Price_Below_Average'])

correlation_price_revenue_multi_seller, product_revenue_comparison_multi_seller.head()


##### 결과분석 
상관계수는 약 -0.06
평균 가격보다 낮은 가격으로 판매하는 것이 반드시 더 높은 Revenue를 생성하지 않음을 의미합니다. 
실제로, 예시 데이터에서도 평균 가격보다 낮게 설정된 가격에서의 평균 Revenue가 평균 가격 또는 그 이상에서 설정된 가격의 평균 Revenue보다 낮은 것을 확인할 수 있습니다.

#### 저렴한 품목을 취급하는 셀러와 고가의 품목을 취급하는 셀러의 Revenue 차이 선형회귀분석

In [ ]:
# import pandas as pd
# import numpy as np
# import random
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression

# 파일을 불러오기
data = pd.read_csv('./data_in/total_df.csv')

# 각 판매자별 총매출(Revenue의 합계)과 평균 가격을 계산
seller_stats = data.groupby('Seller_id').agg(
    Total_Revenue=pd.NamedAgg(column='Revenue', aggfunc='sum'),  # 총 매출
    Average_Price=pd.NamedAgg(column='Price', aggfunc='mean')   # 평균 가격
).reset_index()

# 데이터를 특성(X)과 타겟(y)으로 분리
X = seller_stats[['Average_Price']] # ** 맞는지 확인해봐야겠다 
y = seller_stats['Total_Revenue']

# 데이터를 학습 세트와 테스트 세트로 분리 (학습 80%, 테스트 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random.randint(1, 100))

# 선형 회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 모델 평가: 학습 세트와 테스트 세트에 대한 R² 값 계산
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

# 모델 평가 결과 출력
print("Train R² score:", train_score)
print("Test R² score:", test_score)


##### 결과 분석

1. 약한 선형 관계:
평균 가격과 총 매출 사이에는 매우 약한 선형 관계가 존재합니다. 이는 결정계수(R²) 값이 매우 낮게 나타났음을 의미하며, 평균 가격이 총 매출을 예측하는데 중요한 요소로 작용하지 않는다는 것을 시사합니다.

2. 변수의 다양성 고려:
매출에 영향을 미치는 다른 요소들이 있을 가능성이 높습니다. 예를 들어, 판매량, 판매 지역, 고객 리뷰 점수, 제품의 카테고리, 판매 촉진 활동 등 다양한 변수들이 매출에 영향을 줄 수 있습니다. 이러한 변수들을 추가적으로 고려해 분석하는 것이 좋습니다.

3. 비선형 관계 검토:
선형 모델이 데이터를 충분히 설명하지 못하는 경우, 매출과 가격 사이의 비선형 관계나 상호작용 효과 등을 고려할 필요가 있습니다. 예를 들어, 평균 가격이 특정 수준을 넘어서면 매출 증가율이 달라질 수 있습니다. 이러한 패턴을 찾기 위해 다항 회귀나 다른 고급 분석 기법을 시도해 볼 수 있습니다.

4. 모델의 정교화:
추가 데이터를 확보하거나 다른 유형의 회귀 모델(예: 릿지, 라쏘 회귀)을 적용해 보는 것도 고려해 볼 수 있습니다. 이러한 모델들은 변수의 개수가 많고 상관 관계가 복잡할 때 유용할 수 있습니다.

5. 분석의 확장:
시계열 데이터를 활용하여 시간에 따른 매출 변화를 분석하거나, 판매자별로 시즌별 혹은 프로모션 기간별 매출 차이를 분석하는 것도 통찰력을 제공할 수 있습니다.

이러한 분석을 통해 판매 전략을 개선하고, 보다 정교한 판매 예측 모델을 구축하는 데 도움이 될 수 있습니다. 추가적인 데이터와 다양한 분석 방법을 사용하여 더 깊이 있는 인사이트를 얻을 수 있습니다.

#### 동일 상품을 한 판매자가 싸게 파는게 매출이랑 어떤 관계가 있는지 파악

##### corr() 매서드 사용

###### 1. 저렴한 품목을 취급하는 셀러와 고가의 품목을 취급하는 셀러의 Revenue 차이 파악

In [ ]:
# 판매자셜 평균 판매 가격 계산

average_price_per_seller = order_products_df.groupby('Seller_id').agg(
    Average_Price=('Price', 'mean'),
    # 총 판매 상품 집계
    Total_Sold_Items=('Order_item_id', 'count')
).reset_index()

# 총 매출 데이터와 병합

seller_price_revenue = pd.merge(average_price_per_seller, seller_analysis[['Seller_id', 'Total_Revenue']], on='Seller_id')

# 상위, 하위 임계값 30%로 설정

threshold_top_30 = seller_price_revenue['Average_Price'].quantile(0.70)
threshold_bottom_30 = seller_price_revenue['Average_Price'].quantile(0.30)

# 임계값을 기준으로 판매자 그룹화

top_30_sellers = seller_price_revenue[seller_price_revenue['Average_Price'] >= threshold_top_30]
bottom_30_sellers = seller_price_revenue[seller_price_revenue['Average_Price'] <= threshold_bottom_30]

# 각 그룹의 총 매출과 총 판매 상품 수의 평균을 계산

average_top_30_revenue = top_30_sellers['Total_Revenue'].mean()
average_bottom_30_revenue = bottom_30_sellers['Total_Revenue'].mean()
average_top_30_sold_items = top_30_sellers['Total_Sold_Items'].mean()
average_bottom_30_sold_items = bottom_30_sellers['Total_Sold_Items'].mean()

average_top_30_revenue, average_bottom_30_revenue, average_top_30_sold_items, average_bottom_30_sold_items


결과분석

- 고가의 물건을 파는 셀러 상위 30%의 평균 Revenue: 약 7,446.55
- 저가의 물건을 파는 셀러 하위 30%의 평균 Revenue: 약 1,833.99

- 고가의 물건을 파는 셀러 상위 30%의 평균 판매된 상품 수: 약 23.1 개
- 저가의 물건을 파는 셀러 하위 30%의 평균 판매된 상품 수: 약 35.8 개

###### 2. 취급하는 품목의 가격과 Revenue의 상관관계

In [ ]:
# 취급 품목의 평균 가격과 총 매출간의 상관계수 계산

price_revenue_correlation = seller_price_revenue[['Average_Price', 'Total_Revenue']].corr()

# 상관관계를 히트맵으로 시각화

plt.figure(figsize=(6, 4))
heatmap = sns.heatmap(price_revenue_correlation, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation between Average Selling Price and Total Revenue')
# plt.title('취급 품목의 평균 판매 가격과 총 매출간의 상관계수') ** 히트맵 한글로 할까요 영어로 둘까요
plt.show()

# 상관계수 확인

print(price_revenue_correlation)

결과분석

상품의 가격과 Revenue 사이의 상관계수는 약 0.093. 매우 약한 상관관계.
단순히 취급하는 상품의 가격이 높다 해서 Revenue에 영향을 끼치는 것은 아님.

##### OLS 모델

In [ ]:
# 필요한 라이브러리를 불러옵니다.

# import pandas as pd
# import statsmodels.api as sm

# 데이터를 로드합니다.

file_path = './data_in/total_df.csv'
data = pd.read_csv(file_path)

# 각 제품의 최저 가격을 찾아서 새로운 열로 추가합니다.

data['Min_Price'] = data.groupby('Product_id')['Price'].transform('min')

# 'Price'와 'Min_Price'가 동일한 경우, 즉 가장 낮은 가격으로 판매하는 경우를 표시하는 열을 추가합니다.

data['Is_Min_Price'] = (data['Price'] == data['Min_Price']).astype(int)

# 'Is_Min_Price'를 독립 변수(X)로, 'Revenue'를 종속 변수(y)로 설정합니다.

X = data[['Is_Min_Price']]
y = data['Revenue']

# statsmodels 라이브러리를 사용하여 선형회귀를 수행하기 위해, X에 상수항을 추가합니다.

X = sm.add_constant(X)

# OLS 모델을 생성하고 학습시킵니다.

model = sm.OLS(y, X).fit()

# 회귀분석 결과를 요약하여 출력합니다.

result = model.summary()
result


결과분석


R-squared (결정계수): 0.000으로, 모델이 데이터의 변동성을 거의 설명하지 못하고 있음을 나타냅니다. 이는 '가장 저렴한 가격으로 판매하는지'가 매출 변동성의 설명에 거의 기여하지 않는다는 것을 의미합니다.

Adj. R-squared (조정된 결정계수): 역시 0.000으로, 변수의 수를 고려해도 설명력이 매우 낮습니다.

F-statistic (F 통계량): 21.42로, 이 모델의 예측 변수가 통계적으로 유의미합니다. 즉, '최소 가격 판매 여부'가 매출에 미치는 영향은 통계적으로 유의하다고 할 수 있습니다.

P-value (F-statistic의 유의확률): 3.70e-06으로, 매우 작아서 이 회귀 모델의 예측 변수가 통계적으로 유의미함을 나타냅니다.

계수 해석

const: 상수항은 140.1152입니다. 이는 '최소 가격으로 판매하지 않는 경우'의 평균 매출을 의미합니다.

Is_Min_Price: 계수는 -6.5314로, 가장 저렴한 가격으로 상품을 판매할 경우 매출이 평균적으로 6.5314만큼 감소한다는 것을 나타냅니다.

시사점

영향력의 한계: '가장 저렴한 가격으로 판매하는지 여부'가 매출에 미치는 영향은 통계적으로 유의미하지만, 전체 매출 변동성을 설명하는 데에는 거의 기여하지 않습니다.

가격 경쟁의 복잡성: 최저 가격이 항상 높은 매출을 보장하지는 않습니다. 소비자의 구매 결정은 가격 외에도 다양한 요인에 의해 영향을 받을 수 있으며, 가장 싼 가격이 반드시 매출 증대로 이어지지는 않을 수 있습니다.

추가 분석의 필요성: 다른 변수들(예: 상품의 카테고리, 프로모션, 시즌, 고객 리뷰 등)과의 상호작용을 추가적으로 분석하여, 가격 결정이 매출에 미치는 전체적인 영향을 더 정밀하게 이해할 필요가 있습니다.

가장 저렴한 가격으로 판매하는지 여부'가 매출에 미치는 영향은 통계적으로 유의미하지만, 전체 매출 변동성을 설명하는 데에는 거의 기여하지 않습니다. ** 이게 무슨 말인지~ 

결정계수(R-squared): 이 값은 회귀 모델이 종속 변수(여기서는 매출)의 변동을 얼마나 잘 설명하는지를 수치적으로 나타냅니다. 이 값이 0에 가까울수록 모델이 데이터의 변동을 설명하는 데에 기여하는 바가 매우 낮다는 것을 의미합니다. 즉, 모델이 데이터를 설명하는 데 효과적이지 않다는 것을 나타냅니다.

계수의 유의성(p-value): 이 값은 모델의 예측 변수(여기서는 '가장 저렴한 가격으로 판매하는지 여부')가 통계적으로 유의미한 영향을 미치는지를 검정합니다. p-value가 매우 작다면, 해당 변수가 종속 변수에 영향을 미치는 것이 통계적으로 유의미하다는 것을 의미합니다.

따라서, "가장 저렴한 가격으로 판매하는지 여부'가 매출에 미치는 영향은 통계적으로 유의미하지만, 전체 매출 변동성을 설명하는 데에는 거의 기여하지 않습니다."라는 문장은 다음과 같은 상황을 설명합니다:

가장 저렴한 가격으로 판매하는 것이 매출에 통계적으로 유의미한 영향을 미치기는 하지만 (즉, 이 변수가 매출에 어떤 영향을 주고 있다는 것이 통계적으로 입증됨),

그러나 이 변수 하나만으로는 전체 매출의 변동성을 충분히 설명하지 못합니다. 즉, 매출에 영향을 주는 다른 많은 요인들이 있을 수 있으며, 이 변수만으로는 매출 데이터의 패턴을 충분히 설명할 수 없다는 것입니다.

### 리뷰 ** 리뷰 응답속도와 매출간의 자료를 못 찾아서 찾아서 수정 필요

#### 1. 판매자의 평균 리뷰 점수와 매출의 상관관계

In [ ]:
# 판매 제품과 리뷰 점수에 대한 데이터를 가져오기 위해 order_items, reviews 병합

order_reviews = pd.merge(order_items, review[['Order_id', 'Review_score']], on='Order_id', how='left')

# Seller_id로 그룹화하여 판매자별 평균 리뷰 점수를 계산

seller_reviews = order_reviews.groupby('Seller_id').agg(
    Average_Review_Score=('Review_score', 'mean')
).reset_index()

# 매출 데이터와 병합

seller_reviews_revenue = pd.merge(seller_reviews, seller_analysis[['Seller_id', 'Total_Revenue']], on='Seller_id')

# 평균 리뷰 점수와 총 매출간의 상관계수 계산

review_revenue_correlation = seller_reviews_revenue[['Average_Review_Score', 'Total_Revenue']].corr()

# 상관관계 히트맵으로 시각화

plt.figure(figsize=(6, 4))
heatmap = sns.heatmap(review_revenue_correlation, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation between Average Review Score and Total Revenue')
plt.show()

# 상관계수 출력
print(review_revenue_correlation)


결과분석

판매자의 평균 리뷰 점수와 총 매출 간의 상관계수는 약 -0.03로 연관성이 떨어지는 것을 확인

#### 2. 상품의 리뷰 평점과 상품 판매량의 상관관계

In [ ]:
# `review` 데이터에서 주문당 평균 리뷰 점수 계산

average_review_per_product = review.groupby('Order_id').agg(
    Average_Review_Score=('Review_score', 'mean')
).reset_index()

# `Product_id` 가져오기 위해 `order_items`와 병합
product_reviews = pd.merge(order_items, average_review_per_product, on='Order_id', how='left')

# 상품별 리뷰 점수 계산을 위해 `Product_id`로 그룹화
average_review_per_product = product_reviews.groupby('Product_id').agg(
    Average_Review_Score=('Average_Review_Score', 'mean')
).reset_index()

# 상품 판매량 집계
product_sales_volume = order_items.groupby('Product_id').agg(Sales_Volume=('Order_item_id', 'count')).reset_index()

# 상품 별 평균 리뷰 점수와 병합
	# 이거 `product_sales_volume` 뭔가요 ???
product_reviews_sales = pd.merge(average_review_per_product, product_sales_volume, on='Product_id', how='left')

# 평균 리뷰 점수와 판매량간의 상관계수 계산 `Sales Volume` <- 판매량으로 작성
reviews_sales_correlation = product_reviews_sales[['Average_Review_Score', 'Sales_Volume']].corr()

# 상관관계 히트맵으로 시각화
plt.figure(figsize=(6, 4))
heatmap = sns.heatmap(reviews_sales_correlation, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation between Average Review Score and Sales Volume')
plt.show()

# 상관계수 출력
print(reviews_sales_correlation)

결과분석

상품의 리뷰 평점과 판매량 간의 상관계수는 약 -0.01

#### 추가분석 ： 낮은 리뷰가 달린 후에도 상품을 구매하는 사람이 있는 지

In [ ]:
# 3점 이하 리뷰들 변수로 저장

low_score_reviews = review[review['Review_score'] <= 3]

# 리뷰와의 관계를 보기 위해 `orders` 데이터와 병합

low_score_reviews_orders = pd.merge(low_score_reviews, orders, on='Order_id')

# 낮은 리뷰에도 판매가 이루어 지고 있는 지 확인 위해 `order_items` 데이터와 병합

low_score_product_sales = pd.merge(low_score_reviews_orders, order_items, on='Order_id')

# 가격 정보 확인을 위해 `products` 데이터와 병합

low_score_product_sales = pd.merge(low_score_product_sales, products, on='Product_id')

# 고가의 상품을 변수로 저장하기 위해 임계값 설정

price_threshold = low_score_product_sales['Price'].quantile(0.75)
expensive_low_score_sales = low_score_product_sales[low_score_product_sales['Price'] >= price_threshold]

# 경향을 분석하기 위해 시계열 데이터인 `Order_purchase_timestamp` 컬럼으로 정렬

expensive_low_score_sales_sorted = expensive_low_score_sales.sort_values(by='Order_purchase_timestamp')

# 낮은 리뷰 점수를 받은 이후에도 판매가 이루어 지고 있는 지 확인하기 위해 그룹화

sales_after_low_reviews = expensive_low_score_sales_sorted.groupby(['Product_id', expensive_low_score_sales_sorted['Order_purchase_timestamp'].dt.date]).agg({
    'Order_item_id': 'sum'  # Sum up the quantity sold per product per day
}).reset_index()

# 10개를 추출해 위 과정이 잘 이루어졌는 지 확인

sales_after_low_reviews.head(10)

결과분석

- 상위 ２５％ 가격 상품에 대해 ： 저평가 리뷰 （ ３점 이하）를 받은 이후에도 상품을 구매한 사례가 다수 있음.

- 낮은 리뷰 점수에도 불구하고 고가 상품이 계속해서 팔릴 수 있음을 시사함．

#### 리뷰 평점과 매출의 선형회귀분석

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, r2_score
# import matplotlib.pyplot as plt
# import seaborn as sns
# import random

# 데이터 로드

data_path = './data_in/total_df.csv'
df = pd.read_csv(data_path)

# 필요한 열('Review_score', 'Revenue')만 추출 및 결측치 제거

df = df.dropna(subset=['Review_score'])

# 'Review_score'와 'Revenue' 간의 상관 관계 시각화

plt.figure(figsize=(8, 5))
sns.scatterplot(data=df, x='Review_score', y='Revenue')
plt.title('Review Score vs Revenue')
plt.xlabel('Review Score')
plt.ylabel('Revenue')
plt.show()

# 선형 회귀 모델 준비 및 데이터 분할

X = df[['Review_score']]
y = df['Revenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random.randint(1, 100))

# 모델 학습

model = LinearRegression()
model.fit(X_train, y_train)

# 예측 및 평가

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 회귀 계수, 절편, MSE, R² 출력

print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')
print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')


결과분석

- 회귀 계수(기울기): 약 -5.77
- 절편: 약 159.28
- 평균 제곱 오차(MSE): 약 46,001.72
- 결정 계수(R²): 0.0021

결과에서 보듯, Review_score와 Revenue 사이의 관계는 매우 약하며, 회귀 계수가 음수라는 것은 리뷰 점수가 높아질수록 매출이 오히려 감소하는 경향을 보인다고 해석할 수 있습니다. 하지만, 결정 계수(R²) 값이 매우 낮아 이 변수들 간의 관계를 설명하는 데는 적합하지 않습니다. 이는 리뷰 점수가 매출을 예측하는 데 유의미한 변수가 아닐 수 있음을 시사합니다.



#### 리뷰 응답 시간 ** 했었던 것 같은데 어디있는 지 위치를 못 찾아서 찾아서 추가하면 될 것 같습니다

### 고객 충성도

In [ ]:
# 전체 매출에 대한 고충성도 고객 대 고지출 고객의 영향 평가

# import pandas as pd

# 업로드된 CSV 파일을 불러와 데이터의 내용을 확인

file_path = './data/total_df.csv'
data = pd.read_csv(file_path)

# 데이터프레임의 첫 몇 행과 컬럼들을 표시하여 구조를 이해

data.head(), data.columns

# Total sales 컬럼 추가

data['Total_Sales'] = data['Price'] + data['Freight_value'] # ** Sales가 Revenue를 기준으로 하고 있는게 아니라 Price와 Freight value의 합을 기준으로 하고 있어서 이거 수정할게요

# 고객별 총 매출 계산

total_sales_per_customer = data.groupby('Customer_id')['Total_Sales'].sum()

# 고객별 구매 횟수 계산

loyalty_counts = data.groupby('Customer_id').size()

# 일회성 구매 확인

one_time_purchases = loyalty_counts[loyalty_counts == 1].count()
repeat_purchases = loyalty_counts[loyalty_counts > 1].count()

# 일회성 구매자 비율 계산

one_time_purchase_percentage = one_time_purchases / loyalty_counts.count() * 100

# 고충성도 고객 식별: 여러 번 구매한 고객

high_loyalty_customers = loyalty_counts[loyalty_counts > 1].index
high_loyalty_sales = total_sales_per_customer[total_sales_per_customer.index.isin(high_loyalty_customers)].sum()

# 고지출 고객 식별: 총 지출이 상위 75% 이상인 고객

high_spending_threshold = total_sales_per_customer.quantile(0.75)
high_spending_customers = total_sales_per_customer[total_sales_per_customer > high_spending_threshold].index
high_spending_sales = total_sales_per_customer[total_sales_per_customer.index.isin(high_spending_customers)].sum()

# 전체 매출 계산

total_sales = total_sales_per_customer.sum()

# 고충성도 고객과 고지출 고객이 전체 매출에 기여하는 비율 계산

high_loyalty_sales_percentage = high_loyalty_sales / total_sales * 100
high_spending_sales_percentage = high_spending_sales / total_sales * 100

# 결과 출력

high_loyalty_sales_percentage, high_spending_sales_percentage, one_time_purchase_percentage


결과분석

1. 고지출 고객의 중요성:
고지출 고객이 전체 매출에 기여하는 비율이 높다는 결과는, 이러한 고객층을 대상으로 한 마케팅 전략이 효과적일 수 있음을 의미합니다. 프리미엄 상품, 특별 프로모션, 또는 맞춤형 서비스를 제공하여 이러한 고객의 만족도와 충성도를 높이는 것이 좋습니다.

2. 충성도 높은 고객의 장기적 가치:
반복 구매를 하는 고객들은 비록 전체 매출 기여도는 낮을 수 있지만, 장기적인 관점에서 회사에 지속적인 수익을 제공할 수 있습니다. 이들을 위한 로열티 프로그램, 개인화된 마케팅, 그리고 고객 관계 관리(CRM) 전략을 강화하는 것이 유리합니다.

3. 일회성 구매자의 비율과 관리 전략:
일회성 구매자의 비율이 높다면, 이들을 재구매로 유도할 수 있는 전략을 모색해야 합니다. 예를 들어, 첫 구매 후 할인 쿠폰 제공, 후속 구매를 유도하는 이메일 마케팅, 제품 또는 서비스의 품질 개선 등을 고려할 수 있습니다.

4. 고객 세분화와 타겟팅:
고지출 고객과 충성도 높은 고객 사이에 명확한 구분을 두고, 각 세그먼트에 맞는 타겟 마케팅을 실행하는 것이 중요합니다. 세분화된 데이터 분석을 통해 각 고객 그룹의 특성과 선호를 파악하고, 이에 기반한 맞춤형 전략을 수립해야 합니다.